## PyTorch installation

pip install torch torchvision torchaudio

for more information https://pytorch.org/get-started/locally/, https://pytorch.org/docs/stable/optim.html#torch.optim.Optimizer

pip install numpy --upgrade

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

In [ ]:
print(torch.__version__)

### (0) Preparing data 

In [ ]:
# Loading the data set
T = np.loadtxt('heart.dat')

In [ ]:
print(len(T[:, 13]))

In [ ]:
# Extract the real features
X_numpy = T[:, [0, 3, 4, 7, 9, 11]]

In [ ]:
# Extract and transform the markers
Y_numpy = T[:, 13] - 1
print(Y_numpy)

In [ ]:
# from numpy array to tensor
X = torch.from_numpy(X_numpy.astype(np.float32))
print(X_numpy)
print(X)
print(X.dtype)

In [ ]:
# from numpy array to tensor
Y = torch.from_numpy(Y_numpy.astype(np.float32))
# Y = Y.view(Y.shape[0], 1)
print(Y_numpy)
print(Y)

In [ ]:
# Number of data pairs and features 
m, d = X.shape
print(m, d)

### (1) Splitting our dataset into a train/test split.

Random selection of the indices of the training and test data

In [ ]:
# Share of training data
p = 0.7
data_ind = np.random.permutation(m)
print((np.ceil(p*m)+1))
ind_train = data_ind[:int((np.ceil(p*m)+1))]
ind_test = [i for i in data_ind if i not in ind_train]

In [ ]:
# Training data
X_train = X[ind_train, :]
Y_train = Y[ind_train]
# print(X_train)
# print(Y_train)

In [ ]:
# Test data
X_test = X[ind_test, :]
Y_test = Y[ind_test]
# print(X_test)
# print(Y_test)

### (2) Building the PyTorch Model Class

In [ ]:
class LogisticRegression(torch.nn.Module):
     def __init__(self, input_dim, output_dim):
         super(LogisticRegression, self).__init__()
         self.linear = torch.nn.Linear(input_dim, output_dim)
     def forward(self, x):
         outputs = torch.sigmoid(self.linear(x))
         return outputs

### (3) Initializing the Model

In [ ]:
# assigning some hyper-parameters:
epochs = 1000 # Indicates the number of passes through the entire training dataset the network has completed
input_dim = d 
output_dim = 1 # Single output 
learning_rate = 0.01

In [ ]:
model = LogisticRegression(input_dim, output_dim)

### (4) Initializing the Loss Function and the Optimizer

##### SGD

In [ ]:
# Binary Cross Entropy Loss
criterion_SGD = torch.nn.BCELoss() 

In [ ]:
# SGD: Implements stochastic gradient descent (optionally with momentum)
optimizer_SGD = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [ ]:
# adjusting learning rate
# ExponentialLR: decays the learning rate of each parameter group by gamma every epoch.
scheduler_SGD = torch.optim.lr_scheduler.ExponentialLR(optimizer_SGD, gamma=0.9)

##### Adam

In [ ]:
# Binary Cross Entropy Loss
criterion_Adam = torch.nn.BCELoss() 

In [ ]:
# Adam: Implements Adam algorithm
optimizer_Adam = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
scheduler_Adam = torch.optim.lr_scheduler.ExponentialLR(optimizer_Adam, gamma=0.9)

##### LBFGS

In [ ]:
# Binary Cross Entropy Loss
criterion_LBFGS = torch.nn.BCELoss() 

In [ ]:
# LBFGS: Implements L-BFGS algorithm, heavily inspired by minFunc
optimizer_LBFGS = torch.optim.LBFGS(model.parameters(), lr=learning_rate)

In [ ]:
scheduler_LBFGS = torch.optim.lr_scheduler.ExponentialLR(optimizer_LBFGS, gamma=0.9)

### (5) Train the Model

In [ ]:
print(X_train.shape)
print(Y_train.shape)

In [ ]:
def training_model(criterion, optimizer, scheduler):
    losses = []
    losses_test = []
    Iterations = []
    iter = 0

    for epoch in range(epochs):
    # for epoch in tqdm(range(int(epochs)),desc='Training Epochs'):
        x = X_train
        labels = Y_train
        def closure():
            optimizer.zero_grad() # Setting our stored gradients equal to zero
            outputs = model(X_train)
            loss = criterion(torch.squeeze(outputs), labels) 
            loss.backward() # Computes the gradient of the given tensor w.r.t. the weights/bias
            return loss

        optimizer.step(closure) # Updates weights and biases with the optimizer (SGD)
        scheduler.step()
        # print(scheduler.get_lr())
        # loss = closure()[0]
        # outputs = closure()[1]

        iter+=1
        if iter%1000==0:
            with torch.no_grad():
                # Calculating the loss and accuracy for the test dataset
                correct_test = 0
                total_test = 0
                print(Y_train)
                print(Y_test)
                
                outputs_test = torch.squeeze(model(X_test))
                loss_test = criterion(outputs_test, Y_test)
                print(f'model parameters: {list(model.parameters())}')

                predicted_test = outputs_test.round().detach().numpy()
                print(f'predictied test {predicted_test}')
                
                total_test += Y_test.size(0)
                correct_test += np.sum(predicted_test == Y_test.detach().numpy())
                accuracy_test = 100 * correct_test/total_test
                losses_test.append(loss_test.item())
                print(f"Iteration: {iter}. \nTest - Loss: {loss_test.item()}. Accuracy: {accuracy_test}")
                
                print(outputs_test.size())
                print(Y_test.size())
"""
                # Calculating the loss and accuracy for the train dataset
                total = 0
                correct = 0
                total += Y_train.size(0)
                correct += np.sum(torch.squeeze(outputs).round().detach().numpy() == Y_train.detach().numpy())
                accuracy = 100 * correct/total
                losses.append(loss.item())
                Iterations.append(iter)
"""
                # print(f"Iteration: {iter}. \nTest - Loss: {loss_test.item()}. Accuracy: {accuracy_test}")
                # print(f"Train -  Loss: {loss.item()}. Accuracy: {accuracy}\n")


### (6) Test the Models

SGD

In [ ]:
training_model(criterion_SGD, optimizer_SGD, scheduler_SGD)

Adam

In [ ]:
# training_model(criterion_Adam, optimizer_Adam, scheduler_Adam)

LBFGS

In [ ]:
# training_model(criterion_LBFGS, optimizer_LBFGS, scheduler_LBFGS)